AgeNet/
│
├── environment/            # layer 1 – world & map
│   ├── __init__.py
│   ├── base.py
│   ├── simple.py
│   ├── sumo.py
│   └── geometry.py
│
├── physics/                # layer 2 – physical state & motion
│   ├── __init__.py
│   ├── mobility.py
│   └── state.py
│
├── communication/          # layer 3 – network & interaction
│   ├── __init__.py
│   ├── channel.py
│   ├── radio.py
│   ├── neighbor_finder.py
│   └── reques_policy.py
│
├── learning/               # layer 4 – intelligence & control
│   ├── __init__.py
│   ├── brain.py
│   ├── hamiltonian.py
│   ├── radius_controller.py
│   └── state.py
│
├── core/
│   ├── __init__.py
│   ├── agent.py
│   └── simulator.py
│
├── analysis/               # post-processing & evaluation
│   ├── __init__.py
│   ├── metrics.py
│   ├── topology.py
│   └── visualization.py
│
├── experiments/
│   ├── __init__.py
│   ├── ploter.py
│   └── exporters.py
│
│
└── main.py


In [ ]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple

from AgeNet import environments, physics, Agent, AgentSimulator, AgentsSimulator
from AgeNet import analyses, experiments


In [2]:
L = 45                                  # The length of the simulation box
buildings_type = "random"               # Type of placement of buildings (random or regular)
num_buildings  = 00                     # Number of buildings in environment
num_streets    = 0                      # Number of streets in environment
manual_environment_parameters = [L, buildings_type, num_buildings, num_streets]

sumo_cfg    = "./Sumo/SumoScenario/simulation.sumocfg"
sumo_binary = "/nix/store/gp4hbfbrxg4zzjs7g3nl7vy0rw4jmm2y-sumo-1.22.0/bin/sumo"
step_length = 1.0                       # 
use_gui     = False                     # 
seed        = None                      # 
sumo_environment_parameters = [sumo_cfg, sumo_binary, step_length, use_gui, seed]

N = 100                                 # Number of agents
Alphas = [-0.5, +0.3, 1.0, -1000]       # Hamiltonian constant coefficients
state_dim     = 3                       # Dimensions of the agent state for neural network input
action_dim    = 2                       # Neural network output dimensions (Q values)
learning_rate = 1e-5                    # Learning rate in model training
discount_rate = 0.98                    # The effect of future decisions on current decisions
batch_size    = 20                      # Selected package of replay_memory to perform training
steps_per_train = 10                    # The training function is called once every ... steps
Initialize_model_path = f'./Saved Model/model_H_best_weight.keras'
brain_parameters = [state_dim, action_dim, learning_rate, discount_rate, batch_size, Initialize_model_path]

absorption  = 0.5
base_delay  = 0.01
max_delay   = 1.0
communication_parameters = [absorption, base_delay, max_delay]

moving     = True                       # Enable the move function to agents
requesting = True                       # Enable the sending request function
training   = True                       # Enable each agent model training function
Functions  = [moving, requesting, training]


log_dir = f"Results/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")




### Sumo Environment

In [ ]:
environment = environments.SumoEnvironment(*sumo_environment_parameters)
environment.start()

Agents = []

simulator  = AgentsSimulator(environment, Agents, Functions)
visualizer = analyses.NetworkVisualizer(environment)
result_exporter = experiments.ResultExporter(environment)
result_ploter   = experiments.ResultPloter(log_dir, environment)


In [ ]:
ids, ids_prev = [], []

for step in range(1001):
    environment.step()
    ids      = environment.get_entity_ids()
    new_ids  = list(set(ids) - set(ids_prev))
    del_ids  = list(set(ids_prev) - set(ids))
    ids_prev = ids

    for id in new_ids:
        physical_state = physics.PhysicalState(environment, id)
        agent = Agent(id, Alphas, brain_parameters, communication_parameters, physical_state)
        Agents.append(agent)
        simulator.add_agent(agent, Functions)

    for id in del_ids:
        removed_agent = next((agent for agent in Agents if agent.id == id), None)
        Agents.remove(removed_agent)
        simulator.remove_agent(removed_agent)
    

    simulator.run(Agents, steps_per_train=10)
    
    # -------------------------------------------------------------------------------------------
    if len(ids) > 1:
        result_exporter.collect(step, Agents)
        if step%5 == 0: visualizer.draw(Agents, step)
    print("\rStep: {}".format(step), end="")

environment.close()
if not os.path.exists(log_dir): os.makedirs(log_dir)

anim = visualizer.camera.animate(interval= 120, repeat=True, repeat_delay= 500, blit=False)
result_exporter.export_excel(f'./{log_dir}')
result_ploter.average_result(result_exporter.averaged_results)
anim.save(f'./{log_dir}/test.gif')


### Simple Environment

In [ ]:
environment = environments.SimpleEnvironment(L, buildings_type, num_buildings, num_streets)

Agents = []
for i in range(N): 
    physical_state = physics.PhysicalState(environment, f'{i}')
    movement       = physics.NonMarkovian(environment)
    agent = Agent(f'{i}', Alphas, brain_parameters, communication_parameters, physical_state, movement)
    Agents.append(agent)

simulator  = AgentsSimulator(environment, Agents, Functions)

visualizer = analyses.NetworkVisualizer(environment)
result_exporter = experiments.ResultExporter(environment)
result_ploter   = experiments.ResultPloter(log_dir)


In [ ]:
for step in range(1001):
    simulator.run(Agents, steps_per_train=10)

    # -------------------------------------------------------------------------------------------
    result_exporter.collect(step, Agents)
    if step%5 == 0: visualizer.draw(Agents, step)
    print("\rStep: {}".format(step), end="")

if not os.path.exists(log_dir): os.makedirs(log_dir)

anim = visualizer.camera.animate(interval= 120, repeat=True, repeat_delay= 500, blit=False)
result_exporter.export_excel(f'./{log_dir}')
result_ploter.average_result(result_exporter.averaged_results)
anim.save(f'./{log_dir}/test.gif')


### Tests

In [ ]:
sumo_binary = "/nix/store/gp4hbfbrxg4zzjs7g3nl7vy0rw4jmm2y-sumo-1.22.0/bin/sumo"
env = environments.SumoEnvironment("./Sumo/SumoScenario/simulation.sumocfg", sumo_binary, use_gui=False)
env.start()

T = 2001

for t in range(T):
    env.step()
    ids = env.get_entity_ids()
    for vid in ids:
        pos = env.get_position(vid)
    print(t, '\t', ids)

env.close()

In [17]:
delta_H = -50

x = np.clip(-delta_H / 4, -20, 2)  # flip
x = np.clip(-delta_H * 4, -20, 2)  # send request
p = np.exp(x)
p



7.38905609893065